## Import general packages

In [6]:
import os
from collections import Counter

In [7]:
from collections import defaultdict
def voc_count(corpus):
    d = defaultdict(int)
    for p in corpus:
        for sent in p:
            for t in sent:
                d[t] += 1
    return d

In [8]:
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
import string

#load list of stop words
with open('./snowball_stopwords.txt','rb') as sw:
    stop_words = [line.strip() for line in sw]

#load punctuations 
punctuations = string.punctuation

#extra characters
extra = []

def pre_process_par(par):
    """
    input: 
       a paragraph
    output:
       list of sentences. Each sentence is a list of tokens.
    """
    output = []
    # make the par lowecase
    par = par.lower()
    
    # split sentences
    sent_par = sent_tokenize(par)
    
    # tokenize and clean all sentences
    for sent in sent_par:
        
        #tokenize each sentence
        tokens = word_tokenize(sent)
        
        # remove repetitve words in a sentenece
        tokens = list(set(tokens))
        
        #remove stop words and clean texts
        tokens = [tok for tok in tokens if 
                                          (tok not in stop_words) and 
                                          (tok not in punctuations) and 
                                          (tok not in extra)]
        
        # put it in the output
        output.append(tokens)
    return output

In [9]:
w2v_path = './glove.840B.300d.txt'

import numpy as np
rng = np.random.RandomState(seed=1)

cn = 0
word2vec = {}
with open(w2v_path,'rb') as w2v:
    content = w2v.read().strip()
    for line in content.split('\n'):
        cn +=1
        line = line.strip().split()
        v = line[0]
        
        vector = line[1:]
        vector = np.matrix(vector,dtype='float32')
        
        word2vec[v] = vector

In [10]:
def sentence_connection(sent1, sent2):
    word_connections = []
    for w2 in sent2:
        max_connection = (w2,None,-1)
        try:
            w2_vec = word2vec[w2]
        except:
            word2vec[w2] = rng.uniform(low=-0.2, high=+0.2, size=(300,))
            w2_vec = word2vec[w2]
            
        for w1 in sent1:
            try:
                w1_vec = word2vec[w1]
            except:
                word2vec[w1] = rng.uniform(low=-0.2, high=+0.2, size=(300,))
                w1_vec = word2vec[w1]
        
            # compute the cosine value
            cosine_w2_w1 =  np.abs(1 - spatial.distance.cosine(w2_vec, w1_vec))
            
            if cosine_w2_w1 >= max_connection[-1]:
                max_connection = (w2,w1,cosine_w2_w1)

            # append max_connection to word_connections
            word_connections.append(max_connection)
        
    # pick up the word connection with maximum weight
    output = (None,None,-1)
    for item in word_connections:
        if item[-1] > output[-1]:
            output = item
            
    # return output
    return output

In [11]:
from collections import defaultdict
from copy import deepcopy
from scipy import spatial
def create_graph(list_of_sents):
    
    # get the number of sentences or nodes
    n = len(list_of_sents)
    
    #initialize the adjacent list for the graph representation of this text
    adj_list = [[] for i in range(n)]

    #for each sentence compute its weight with each previouse sentence   
    for i in range(1, n):            
            sent_i = list_of_sents[i]
            
            for j in range(0,i):
                sent_j = list_of_sents[j]
                

                # compute the connection between sent_i and sent_j sen_j preceds sent_i
                edge_ij = sentence_connection(sent_j, sent_i)
                
                
                # check for the threshold
                weight = edge_ij[-1]
                #if weight>=0.9:
                adj_list[i].append((j,weight))
                

    # return adj_list
    return adj_list
        
    

In [12]:
dump_text = "Barak is the US president . Angela is the chancellor of Germany . Barak and Angela met each other in France ."
#dump_text = "Mohsen likes briliant people . Ali tries hard . Mohsen is a successful mohsen person . Mohsen is smart . "
dump_text = pre_process_par(dump_text)
print dump_text

#sentence_connection(dump_text[1],dump_text[2])

adj_list = create_graph(dump_text)
#print word_connections
g = adj_list
for k, v in enumerate(g):
    print "%d -->  %s"%(k,v)
#x = word2vec['angela'].shape
#np.abs(1 - spatial.distance.cosine(x, x))

[['us', 'barak', 'president'], ['chancellor', 'angela', 'germany'], ['angela', 'france', 'met', 'barak']]
0 -->  []
1 -->  [(0, 0.47426614165306091)]
2 -->  [(0, 1.0), (1, 1.0)]


In [13]:
def save(graph_set, path, weighted=False):
    output_content = []
    for graph_name,  adj_list in enumerate(graph_set):
        output_content.append('XP')
        output_content.append('% '+str(graph_name))
        #output_content.append('t # %d'%graph_name)
        num_nodes = len(adj_list)

        for n in range(num_nodes):
            output_content.append('v %s a'%str(n))

        for i, edges in enumerate(adj_list):
            if i>0:
                for j in edges:
                    #Note: we computed edges backward, but we should 
                    # save them forward to be compatible with NAACL16
                    source = j[0] 
                    target = i
                    if target > source:
                        if source>0 and target>0:
                            if weighted:
                                output_content.append('d %s %s %s'%(str(source),str(target),str(j[1])))
                            else:
                                output_content.append('d %s %s 1'%(str(source),str(target)))
                    else:
                        raise ValueError("Backward eadge?")

    with open(path,'wb') as out:
        out.write('\n'.join(output_content))

In [14]:
import codecs
with codecs.open('./texts', 'rb','utf8') as orig:
    orig_paragraphs = []
    for line in orig:
        if line != '\n' and len(line)>0:
            orig_paragraphs.append(line.strip())
print "# paragraphs in original: %d"%len(orig_paragraphs)

# paragraphs in original: 3


In [15]:
from joblib import Parallel, delayed
original_pars = []
original_pars = Parallel(n_jobs=-1, verbose=-1, backend="multiprocessing")(
             map(delayed(pre_process_par), orig_paragraphs))

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished


In [16]:
from joblib import Parallel, delayed
graphs = []
graphs = Parallel(n_jobs=-1, verbose=-1, backend="multiprocessing")(
                 map(delayed(create_graph), original_pars))

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   32.2s finished


In [ ]:
# import pickle
# with open('original_pars_graphs_lcg.pickle','wb') as out:
#     pickle.dump(graphs, out)   

In [17]:
save(graphs,'./graph_set.g', weighted=True)